# Rainbow options with Integration & Direct Amplitude Loading

## Data definitions

In [2]:
import numpy as np

MAX_FRACTION_PLACES = 1

K = 190
S0 = [193.97, 189.12]
dt = 250

COV = np.array([[0.000335, 0.000257], [0.000257,0.000418]])
SIGMA = np.array([0.01817985, 0.02035118])
MU_LOG_RET = np.array([0.00050963, 0.00062552])

MU = MU_LOG_RET*dt


CHOLESKY = np.linalg.cholesky(COV)*np.sqrt(dt)
SCALING_FACTOR = 1 / CHOLESKY[0,0]


In [3]:
from classiq import FunctionLibrary
rainbow_option_library = FunctionLibrary()

## Gaussian State preparation

In [6]:
import scipy

def gaussian_discretization(num_qubits, mu=0, sigma=1, stds_around_mean_to_include=3):
    lower = mu - stds_around_mean_to_include * sigma
    upper = mu + stds_around_mean_to_include * sigma
    num_of_bins = 2**num_qubits
    sample_points = np.linspace(lower, upper, num_of_bins + 1)

    def single_gaussian(x: np.ndarray, _mu: float, _sigma: float) -> np.ndarray:
        cdf = scipy.stats.norm.cdf(x, loc=_mu, scale=_sigma)
        return cdf[1:] - cdf[0:-1]

    non_normalized_pmf = single_gaussian(sample_points, mu, sigma),
    real_probs = non_normalized_pmf / np.sum(non_normalized_pmf)
    return sample_points[:-1], real_probs 

In [7]:
import matplotlib.pyplot as plt

NUM_QUBITS=2
NUM_ASSETS=2

grid_points, probabilities = gaussian_discretization(NUM_QUBITS)
#plt.scatter(grid_points, probabilities)

In [8]:
from classiq.builtin_functions import StatePreparation

sp_params = StatePreparation(probabilities=probabilities, num_qubits=NUM_QUBITS, error_metric={"KL": {"upper_bound": 0}})

In [9]:
STEP_X = grid_points[1] - grid_points[0]
MIN_X = grid_points[0]

### SANITY CHECK

The process must be stopped if the strike price $K$ is greater than the maximum value reacheable by the assets during the simulation, to avoid meaningless results. The payoff is $0$ in this case, so there is no need to simulate.

In [10]:
from IPython.display import Markdown

if K >= max(S0*np.exp(np.dot(CHOLESKY,[grid_points[-1]]*2) + MU)):
    display(Markdown('<font color=\'red\'> K always greater than the maximum asset values. Stop the run, the payoff is 0</font>'))

### CLASSICAL SIMULATION

In [11]:
from itertools import product
binary_combinations = list(product(range(2**NUM_QUBITS),repeat=2))

In [12]:
import pandas as pd

simulation = pd.DataFrame.from_dict({"quantum_samples": list(range(len(grid_points))) ,"classical_samples": grid_points, "probabilities": probabilities[0]} )

In [13]:
sim = pd.DataFrame(binary_combinations)
sim = sim.merge(simulation, left_on=0, right_on="quantum_samples")
sim = sim.merge(simulation, left_on=1, right_on="quantum_samples")

In [14]:
sim["probabilities_xy"] = sim["probabilities_x"]*sim["probabilities_y"]
sim["perturbed_sample_x"] = sim["classical_samples_x"]*CHOLESKY[0,0]
sim["perturbed_sample_y"] = (sim["classical_samples_x"]*CHOLESKY[1,0]) + (sim["classical_samples_y"]*CHOLESKY[1,1])
sim["ret_x"] = sim["perturbed_sample_x"] + MU[0]
sim["ret_y"] = sim["perturbed_sample_y"] + MU[1]
sim["exp_x"] = np.exp(sim["ret_x"])
sim["exp_y"] = np.exp(sim["ret_y"])
sim["asset_x"] = sim["exp_x"]*S0[0]
sim["asset_y"] = sim["exp_y"]*S0[1]
sim["max_asset"] = sim[["asset_x", "asset_y"]].max(axis=1)
sim["payoff"] = sim["max_asset"] - K
sim.loc[sim["payoff"]<0, "payoff" ] = 0

In [15]:
sim

,0,1,quantum_samples_x,classical_samples_x,probabilities_x,quantum_samples_y,classical_samples_y,probabilities_y,probabilities_xy,perturbed_sample_x,perturbed_sample_y,ret_x,ret_y,exp_x,exp_y,asset_x,asset_y,max_asset,payoff
0,0,0,0,-3.0,0.065635,0,-3.0,0.065635,0.004308,-0.868188,-1.370945,-0.740780,-1.214565,0.476742,0.296839,92.473604,56.138218,92.473604,0.000000
1,1,0,1,-1.5,0.434365,0,-3.0,0.065635,0.028509,-0.434094,-1.037924,-0.306686,-0.881544,0.735881,0.414143,142.738905,78.322750,142.738905,0.000000
2,2,0,2,0.0,0.434365,0,-3.0,0.065635,0.028509,0.000000,-0.704902,0.127407,-0.548522,1.135880,0.577803,220.326604,109.274099,220.326604,30.326604
3,3,0,3,1.5,0.065635,0,-3.0,0.065635,0.004308,0.434094,-0.371881,0.561501,-0.215501,1.753303,0.806137,340.088165,152.456707,340.088165,150.088165
4,0,1,0,-3.0,0.065635,1,-1.5,0.434365,0.028509,-0.868188,-1.018494,-0.740780,-0.862114,0.476742,0.422269,92.473604,79.859433,92.473604,0.000000
5,1,1,1,-1.5,0.434365,1,-1.5,0.434365,0.188673,-0.434094,-0.685472,-0.306686,-0.529092,0.735881,0.589139,142.738905,111.418043,142.738905,0.000000
6,2,1,2,0.0,0.434365,1,-1.5,0.434365,0.188673,0.000000,-0.352451,0.127407,-0.196071,1.135880,0.821954,220.326604,155.447890,220.326604,30.326604
7,3,1,3,1.5,0.065635,1,-1.5,0.434365,0.028509,0.434094,-0.019430,0.561501,0.136950,1.753303,1.146771,340.088165,216.877318,340.088165,150.088165
8,0,2,0,-3.0,0.065635,2,0.0,0.434365,0.028509,-0.868188,-0.666043,-0.740780,-0.509663,0.476742,0.600698,92.473604,113.604052,113.604052,0.000000
9,1,2,1,-1.5,0.434365,2,0.0,0.434365,0.188673,-0.434094,-0.333021,-0.306686,-0.176641,0.735881,0.838080,142.738905,158.497759,158.497759,0.000000


## Max operation

In [31]:
from classiq.builtin_functions import Arithmetic
from classiq import RegisterUserInput
from functools import reduce


def get_affine_expr(i):
    return reduce(lambda x, y: f"{x}+{y}", [f"x{j} * {SCALING_FACTOR * CHOLESKY[i,j]}" for j in range(NUM_ASSETS) if CHOLESKY[i,j]])

max_params = Arithmetic( 
    expression = f"max({get_affine_expr(0)}, {get_affine_expr(1)} + c)",
    definitions=dict(
        x0 = RegisterUserInput(size=NUM_QUBITS),
        x1 = RegisterUserInput(size=NUM_QUBITS),
        c = SCALING_FACTOR * (np.log(S0[1]) + MU[1] - (np.log(S0[0]) + MU[0]) + MIN_X*sum(CHOLESKY[1] - CHOLESKY[0]))/(STEP_X),
    ),
    uncomputation_method="optimized",
    inputs_to_save = ['x0', 'x1'],
    machine_precision=MAX_FRACTION_PLACES
)

In [32]:
frac_places = max_params.outputs['expression_result'].fraction_places
al_num_qubits = max_params.outputs['expression_result'].size

### CLASSICAL SIMULATION

We perform the classical simulation by considering the error of the aritmetic, considering the binning in the following way:
- Always floor to the positive number:\
    $1.99 \rightarrow 1.5$
- Treat the negative number as a positive one:\
    $-1.99 \rightarrow 1.99 \rightarrow 1.5 \rightarrow -1.5$

\*examples with 1 fraction place

In [33]:
import math

def round_to_frac(num, frac_places):
    sign = 1
    if num < 0:
        sign = -1
        num = num * sign
    precision = 1/(2**frac_places)
    dig = num%1
    num_new = num - dig
    return (num_new + (math.floor(dig/precision)*precision))*sign

In [34]:
approximated_simulation = sim.iloc[:,:9].copy()
#ax+by+c

approximated_simulation["zx_a"] = CHOLESKY[0,0]*SCALING_FACTOR
approximated_simulation["zy_a"] = CHOLESKY[1,0]*SCALING_FACTOR
approximated_simulation["zy_b"] = SCALING_FACTOR*CHOLESKY[1,1]
approximated_simulation["zy_c"] = SCALING_FACTOR * (np.log(S0[1]) + MU[1] - (np.log(S0[0]) + MU[0]) + MIN_X*sum(CHOLESKY[1] - CHOLESKY[0]))/(STEP_X)
approximated_simulation[["approx_zx_a","approx_zy_a", "approx_zy_b", "approx_zy_c"]]= approximated_simulation[["zx_a","zy_a", "zy_b", "zy_c"] ].applymap(lambda x : round_to_frac(x,frac_places))
approximated_simulation["zx"] = approximated_simulation["approx_zx_a"]*approximated_simulation["quantum_samples_x"]
approximated_simulation["zy"] = (approximated_simulation["approx_zy_a"]*approximated_simulation["quantum_samples_x"])+(approximated_simulation["approx_zy_b"]*approximated_simulation["quantum_samples_y"])+ approximated_simulation["approx_zy_c"]
approximated_simulation["z"] = approximated_simulation[["zx", "zy"]].max(axis=1)
approximated_simulation["max_asset"] = S0[0] * np.exp((STEP_X / SCALING_FACTOR ) * approximated_simulation["z"] + (MU[0]+MIN_X*CHOLESKY[0].sum()))

In [41]:
mask = sim["max_asset"] != approximated_simulation["max_asset"]
sim[mask]

,0,1,quantum_samples_x,classical_samples_x,probabilities_x,quantum_samples_y,classical_samples_y,probabilities_y,probabilities_xy,perturbed_sample_x,perturbed_sample_y,ret_x,ret_y,exp_x,exp_y,asset_x,asset_y,max_asset,payoff
8,0,2,0,-3.0,0.065635,2,0.0,0.434365,0.028509,-0.868188,-0.666043,-0.740780,-0.509663,0.476742,0.600698,92.473604,113.604052,113.604052,0.000000
9,1,2,1,-1.5,0.434365,2,0.0,0.434365,0.188673,-0.434094,-0.333021,-0.306686,-0.176641,0.735881,0.838080,142.738905,158.497759,158.497759,0.000000
10,2,2,2,0.0,0.434365,2,0.0,0.434365,0.188673,0.000000,0.000000,0.127407,0.156380,1.135880,1.169270,220.326604,221.132426,221.132426,31.132426
12,0,3,0,-3.0,0.065635,3,1.5,0.065635,0.004308,-0.868188,-0.313591,-0.740780,-0.157211,0.476742,0.854523,92.473604,161.607467,161.607467,0.000000
13,1,3,1,-1.5,0.434365,3,1.5,0.065635,0.028509,-0.434094,0.019430,-0.306686,0.175810,0.735881,1.192211,142.738905,225.471018,225.471018,35.471018
14,2,3,2,0.0,0.434365,3,1.5,0.065635,0.028509,0.000000,0.352451,0.127407,0.508831,1.135880,1.663346,220.326604,314.571975,314.571975,124.571975
15,3,3,3,1.5,0.065635,3,1.5,0.065635,0.004308,0.434094,0.685472,0.561501,0.841852,1.753303,2.320662,340.088165,438.883580,438.883580,248.883580


In [42]:
approximated_simulation

,0,1,quantum_samples_x,classical_samples_x,probabilities_x,quantum_samples_y,classical_samples_y,probabilities_y,probabilities_xy,zx_a,...,zy_b,zy_c,approx_zx_a,approx_zy_a,approx_zy_b,approx_zy_c,zx,zy,z,max_asset
0,0,0,0,-3.0,0.065635,0,-3.0,0.065635,0.004308,1.0,...,0.811924,-1.149766,1.0,0.5,0.5,-1.0,0.0,-1.0,0.0,92.473604
1,1,0,1,-1.5,0.434365,0,-3.0,0.065635,0.028509,1.0,...,0.811924,-1.149766,1.0,0.5,0.5,-1.0,1.0,-0.5,1.0,142.738905
2,2,0,2,0.0,0.434365,0,-3.0,0.065635,0.028509,1.0,...,0.811924,-1.149766,1.0,0.5,0.5,-1.0,2.0,0.0,2.0,220.326604
3,3,0,3,1.5,0.065635,0,-3.0,0.065635,0.004308,1.0,...,0.811924,-1.149766,1.0,0.5,0.5,-1.0,3.0,0.5,3.0,340.088165
4,0,1,0,-3.0,0.065635,1,-1.5,0.434365,0.028509,1.0,...,0.811924,-1.149766,1.0,0.5,0.5,-1.0,0.0,-0.5,0.0,92.473604
5,1,1,1,-1.5,0.434365,1,-1.5,0.434365,0.188673,1.0,...,0.811924,-1.149766,1.0,0.5,0.5,-1.0,1.0,0.0,1.0,142.738905
6,2,1,2,0.0,0.434365,1,-1.5,0.434365,0.188673,1.0,...,0.811924,-1.149766,1.0,0.5,0.5,-1.0,2.0,0.5,2.0,220.326604
7,3,1,3,1.5,0.065635,1,-1.5,0.434365,0.028509,1.0,...,0.811924,-1.149766,1.0,0.5,0.5,-1.0,3.0,1.0,3.0,340.088165
8,0,2,0,-3.0,0.065635,2,0.0,0.434365,0.028509,1.0,...,0.811924,-1.149766,1.0,0.5,0.5,-1.0,0.0,0.0,0.0,92.473604
9,1,2,1,-1.5,0.434365,2,0.0,0.434365,0.188673,1.0,...,0.811924,-1.149766,1.0,0.5,0.5,-1.0,1.0,0.5,1.0,142.738905


In [43]:
approximated_simulation[mask]

,0,1,quantum_samples_x,classical_samples_x,probabilities_x,quantum_samples_y,classical_samples_y,probabilities_y,probabilities_xy,zx_a,...,zy_b,zy_c,approx_zx_a,approx_zy_a,approx_zy_b,approx_zy_c,zx,zy,z,max_asset
8,0,2,0,-3.0,0.065635,2,0.0,0.434365,0.028509,1.0,...,0.811924,-1.149766,1.0,0.5,0.5,-1.0,0.0,0.0,0.0,92.473604
9,1,2,1,-1.5,0.434365,2,0.0,0.434365,0.188673,1.0,...,0.811924,-1.149766,1.0,0.5,0.5,-1.0,1.0,0.5,1.0,142.738905
10,2,2,2,0.0,0.434365,2,0.0,0.434365,0.188673,1.0,...,0.811924,-1.149766,1.0,0.5,0.5,-1.0,2.0,1.0,2.0,220.326604
12,0,3,0,-3.0,0.065635,3,1.5,0.065635,0.004308,1.0,...,0.811924,-1.149766,1.0,0.5,0.5,-1.0,0.0,0.5,0.5,114.889429
13,1,3,1,-1.5,0.434365,3,1.5,0.065635,0.028509,1.0,...,0.811924,-1.149766,1.0,0.5,0.5,-1.0,1.0,1.0,1.0,142.738905
14,2,3,2,0.0,0.434365,3,1.5,0.065635,0.028509,1.0,...,0.811924,-1.149766,1.0,0.5,0.5,-1.0,2.0,1.5,2.0,220.326604
15,3,3,3,1.5,0.065635,3,1.5,0.065635,0.004308,1.0,...,0.811924,-1.149766,1.0,0.5,0.5,-1.0,3.0,2.0,3.0,340.088165


In [44]:
approximated_simulation["payoff"] = approximated_simulation["max_asset"] - K
approximated_simulation.loc[approximated_simulation["payoff"]<0, "payoff" ] = 0

In [46]:
sum(approximated_simulation["payoff"]*approximated_simulation["probabilities_xy"])

23.023792399612088

## Comparator

In [51]:
a = (STEP_X / SCALING_FACTOR)
b = np.log(S0[0]) + MU[0]+MIN_X*CHOLESKY[0].sum()
     
COMP_VALUE = (np.log(K) - b) / a

comp_params = Arithmetic(
    expression="a>=b",
    definitions=dict(
        a=RegisterUserInput(size=al_num_qubits, fraction_places=frac_places),
        b=COMP_VALUE,
    ),
    target=RegisterUserInput(size=1),
    inputs_to_save=["a"]
)

### Integration Comparator

In [56]:
integration_params = Arithmetic( 
    expression = "a <= b",
    definitions=dict(
        a = RegisterUserInput(size=al_num_qubits),
        b = RegisterUserInput(size=al_num_qubits),
    ),
    uncomputation_method="optimized",
    target=RegisterUserInput(size=1),
    inputs_to_save = ['a', 'b'],
)

## Integration Method 2

In [59]:
exp_rate = (1/(2**frac_places)) * a
B = (np.exp((2**al_num_qubits)*exp_rate) -1)/np.exp(exp_rate)
A = 1 / np.exp(exp_rate)
C = S0[0]*np.exp((MU[0]+MIN_X*CHOLESKY[0].sum()))

In [63]:
from classiq.builtin_functions import ExponentialStatePreparation, RYGate

exp_sp_params_method2 = ExponentialStatePreparation(num_qubits = al_num_qubits, rate = -exp_rate)
ry_params_method2 = RYGate(theta=2* np.arcsin(np.sqrt( (K- (C*A) )/(C*B) )))

In [64]:
from classiq import QReg, FunctionLibrary, FunctionGenerator, QUInt, ControlState

#rainbow_option_library = FunctionLibrary()
fg = FunctionGenerator("rainbow_integration_method2")

inputs_dict = fg.create_inputs({'IN': QUInt[NUM_ASSETS * NUM_QUBITS + al_num_qubits + 1]})

asset_regs = []
for i in range(NUM_ASSETS):
    asset_regs.append(fg.StatePreparation(sp_params, strict_zero_ios=False, in_wires=inputs_dict['IN'][i * NUM_QUBITS: (i+1) * NUM_QUBITS])['OUT'])
    
max_output = fg.Arithmetic(max_params, should_control=False, in_wires={"x0": asset_regs[0], "x1": asset_regs[1]})
comp_output = fg.Arithmetic(comp_params, should_control=False, in_wires={'a':max_output['expression_result']})

ctrl_al = ControlState(ctrl_state='1', name="CTRL")
ctrl_ry = ControlState(ctrl_state='0', name="CTRL")

exp_sp = fg.ExponentialStatePreparation(exp_sp_params_method2, strict_zero_ios=False,
                             in_wires={'IN':inputs_dict['IN'][NUM_ASSETS * NUM_QUBITS: NUM_ASSETS * NUM_QUBITS + al_num_qubits]})

integration_out = fg.Arithmetic(integration_params, control_states=ctrl_al, 
                                in_wires={"a": exp_sp['OUT'], 'b': comp_output['a'], "arithmetic_target": inputs_dict["IN"][-1],
                                          'CTRL': comp_output['expression_result']})

ry_out = fg.RYGate(ry_params_method2, control_states=ctrl_ry,
                   in_wires={'TARGET': integration_out['expression_result'],
                             'CTRL': integration_out['CTRL']})


# Uncompute
comp_output = fg.Arithmetic(comp_params, is_inverse=True, should_control=False, release_by_inverse=True,
                            in_wires={'a':integration_out["b"], "expression_result": ry_out['CTRL']})
inv_max_output = fg.Arithmetic(max_params, is_inverse=True, should_control=False, release_by_inverse=True,
                               in_wires={"expression_result": comp_output["a"], 'x0': max_output['x0'], 'x1': max_output['x1']})

fg.set_outputs({"OUT" : QReg.concat(inv_max_output['x0'], inv_max_output['x1'], integration_out['a'], ry_out['TARGET'])})

rainbow_option_library.add_function(fg.to_function_definition(), True)

# Create the iterative amplitude estimation circuit

In [68]:
from classiq.builtin_functions import GroverOperator, ArithmeticOracle, AmplitudeEstimation
from classiq import RegisterUserInput

RAINBOW_IMPLEMENTATION = 'rainbow_integration_method2'

rainbow = rainbow_option_library.get_function(RAINBOW_IMPLEMENTATION)

aritmeticOracle =  ArithmeticOracle(
    expression=f"((a>>{rainbow.output_decls['OUT'].size-1})%2==1)",
    definitions=dict(
        a=RegisterUserInput(size=rainbow.output_decls['OUT'].size),
    ),
    uncomputation_method="optimized"
)

grover_operator_params = GroverOperator(
    oracle_params = aritmeticOracle,
    state_preparation =RAINBOW_IMPLEMENTATION,
    state_preparation_params = rainbow,
)

Here we create a state preperation, followed by "k" repetitions of the grover's operator.

In [69]:
from classiq import Model, Constraints, Preferences, synthesize
from classiq.execution import QaeWithQpeEstimationMethod
        
model = Model()

model.include_library(rainbow_option_library)

A_out = model.apply(RAINBOW_IMPLEMENTATION)

grover_out = model.GroverOperator(grover_operator_params, in_wires={"a": A_out['OUT']}, 
                                  power="k") # set parametric number of repetitions, to be changed during the IQAE execution

Notice that the `iqae` execution requires only one output from the circuit, which is the auxiliary target qubit.

In [70]:
model.set_outputs({'ind': grover_out['a'][-1]})
model.iqae(epsilon=0.01, alpha=0.05)

In [71]:
model.constraints = Constraints(max_width=24)
model.preferences = Preferences(timeout_seconds=60*20)
qprog = synthesize(model.get_model())

In [72]:
from classiq import show
show(qprog)

Opening: https://platform.classiq.io/circuit/6dbd26ef-3a31-49b3-b886-4703e3476af8?version=0.35.0


In [73]:
from classiq import execute, set_quantum_program_execution_preferences
from classiq.execution import ExecutionPreferences
from classiq.interface.backend.backend_preferences import IBMBackendPreferences, BackendPreferences

backend_preferences = IBMBackendPreferences(
    backend_service_provider="IBM Quantum", backend_name="ibmq_qasm_simulator", access_token="9b5c2337e9f3953a1e3d0c68a8fb8cec49dea771582d6c79f9054cc9e910cf6157fe1401b65da058e3fc56b7f2fadad371069a9cf9d00ec3a4f731ecd61fc875"
)

# Define execution preferences
execution_preferences = ExecutionPreferences(
    num_shots=1000, # The number of shots is a hyperparameter of the algorithm
    backend_preferences=backend_preferences
)

# Set the execution preferences
qprog = set_quantum_program_execution_preferences(qprog, execution_preferences)

qae_result = execute(qprog).result()

In [75]:
res = qae_result[0].value
print(res.estimation, res.confidence_interval)

0.05947294142802455 (0.058763768767926626, 0.06018211408812247)


### post process for integration method2

In [76]:
option_value = (res.estimation * (C*B)) + (C*A) -K
confidence_interval = (np.array(res.confidence_interval) * (C*B)) + (C*A) - K
print(option_value, confidence_interval)

22.654556227296325 [21.00634167 24.30277078]
